In [1]:
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL, urllib
import os
from PIL import Image
import pandas as pd

In [2]:
def get_diagnosis(image_name, file):

    matching_row = file[file['image_id'] == image_name.split('.')[0]]
    if not matching_row.empty:
        return matching_row.iloc[0]['dx']
    return None

def create_image_diagnosis_dict(base_path, folders, file):
    metadata = pd.read_csv(file)

    image_diagnosis_dict = {}
    for folder in folders:
        path = base_path + folder
        for image_name in os.listdir(path):
            diagnosis = get_diagnosis(image_name, metadata)
            if diagnosis is not None:
                # Add the image name and its diagnosis to the dictionary
                image_diagnosis_dict[image_name] = diagnosis
    return image_diagnosis_dict

In [3]:
class diagnoses(Dataset):
        CLASSES = ('nv', 'akiec', 'bcc', 'bkl', 'df', 'mel', 'vasc')  # Class variable

        def __init__(self, data, transform=None):
            self.data = data
            self.transform = transform

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            image_path, label = self.data[idx]
            image = Image.open(image_path)
            if self.transform:
                image = self.transform(image)
            try:
                label_idx = self.CLASSES.index(label) if isinstance(label, str) else label
            except ValueError:
                print(f"Error with label: {label} at index: {idx}")  # This will indicate the problematic label
                raise  # Re-raise the exception to halt the execution
            return image, label_idx

In [4]:
base_path = 'C:\\Users\\krish\\APS360\\PROJECT'
folders = ('\\HAM10000_images_part_1', '\\HAM10000_images_part_2')
metadata = '\\HAM10000_metadata.csv'

dataset = []

matches = create_image_diagnosis_dict(base_path, folders, base_path + metadata)

In [5]:
def make_dataset(idx, base_path, folders, max_images=None):
    CLASSES = ('nv', 'akiec', 'bcc', 'bkl', 'df', 'mel', 'vasc')  # Class variable
    dataset = []
    image_count = 0
    for folder in folders:
        path = base_path + folder
        for img in sorted(os.listdir(path)):  # Optionally sort to have consistent results
            if max_images and image_count >= 200000:
                break  # Stop adding images once the maximum is reached
            image_path = os.path.join(path, img)
            label = idx.get(img)
            if label is None or label not in CLASSES:
                continue
            dataset.append((image_path, label))
            image_count += 1
        if max_images and image_count >= max_images:
            break  # Ensure to break the outer loop as well
    return dataset

In [6]:
def get_data_loader(batch_size, idx, base_path, folders):

    transform = transforms.Compose([
        transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    dataset = make_dataset(idx, base_path, folders, 50000)

    training_validation_ratio = 0.7
    split = int(len(dataset) * training_validation_ratio) #split at 70%

    validation_set = dataset[split:]
    train_set = dataset[:split]

    #declare pytorch dataset
    torch_train = diagnoses(train_set, transform)
    torch_validation = diagnoses(validation_set, transform)


    #create loaders
    train_loader = torch.utils.data.DataLoader(torch_train, batch_size=batch_size,
                                               num_workers=0, shuffle = True)#change num workers later
    val_loader = torch.utils.data.DataLoader(torch_validation, batch_size=batch_size,
                                              num_workers=0, shuffle = False)#change num workers later
    print("get data loader complete")

    return train_loader, val_loader

In [7]:
train_loader, val_loader = get_data_loader(64, matches, base_path, folders)

get data loader complete


In [8]:
len(train_loader)

110

In [9]:
import torch
from torchvision import datasets, transforms

data_iter = iter(train_loader)
images, labels = next(data_iter)

print(f'Batch shape: {images.shape}')

Batch shape: torch.Size([64, 3, 128, 128])


In [10]:
def get_model_name(name, batch_size, learning_rate, epoch):

    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

def evaluate(net, loader, criterion):
    net.eval()  # Set the network to evaluation mode
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0
    
    with torch.no_grad():  # Disable gradient computation
        for data in loader:
            inputs, labels = data
            
            # Move inputs and labels to GPU if CUDA is available
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
            
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total_err += (predicted != labels).sum().item()
            total_loss += loss.item()
            total_epoch += labels.size(0)
    
    # Calculate average error and loss
    avg_err = total_err / total_epoch
    avg_loss = total_loss / len(loader)
    net.train()  # Set the network back to training mode
    return avg_err, avg_loss

In [11]:
#add data % as an attribute, add necessary attributes for data loading
#incorporate data loading functions inside the training function

def train_net(net, train_loader, val_loader, batch_size=64, learning_rate=0.01, num_epochs=30, step_size=2):
    # Fixed PyTorch random seed for reproducible result
    torch.manual_seed(1000)

    # The loss function will be Cross Entropy Loss (multi-class classification)
    # Optimizer will be SGD with Momentum.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

    # Set up some numpy arrays to store the training/test loss/erruracy
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)

    # Train the network
    start_time = time.time()
    for epoch in range(num_epochs):
        net.train()  # Set the network to training mode
        print(f"{epoch} initiated")
        total_train_loss = 0.0
        total_train_err = 0.0
        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            print(f"{i} batch initiated")
            inputs, labels = data

            # Enable GPU usage if available
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total_train_err += (predicted != labels).sum().item()
            total_train_loss += loss.item()
            total_epoch += labels.size(0)
            
        
        train_err[epoch] = total_train_err / total_epoch
        train_loss[epoch] = total_train_loss / (i+1)
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
        print(f"Epoch {epoch + 1}: Train err: {train_err[epoch]}, Train loss: {train_loss[epoch]} | "
              f"Validation err: {val_err[epoch]}, Validation loss: {val_loss[epoch]}")
        
        # Save the current model (checkpoint) to a file
        model_path = f"{net.name}_batch_{batch_size}_lr_{learning_rate}_epoch_{epoch}.model"
        torch.save(net.state_dict(), model_path)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total time elapsed: {elapsed_time:.2f} seconds")

    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)
    np.savetxt(f"{model_path}_train_err.csv", train_err)
    np.savetxt(f"{model_path}_train_loss.csv", train_loss)
    np.savetxt(f"{model_path}_val_err.csv", val_err)
    np.savetxt(f"{model_path}_val_loss.csv", val_loss)

In [12]:
def plot_training_curve(path):
    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))

    plt.figure(figsize=(4, 4))

    plt.title("Train vs Validation Error")
    n = len(train_err) # number of epochs
    plt.plot(range(1, n+1), train_err, label="Train")
    plt.plot(range(1, n+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.ylim(0, 1)
    plt.legend(loc='best')
    plt.show()

    plt.figure(figsize=(4, 4))

    plt.title("Train vs Validation Loss")
    plt.plot(range(1, n+1), train_loss, label="Train")
    plt.plot(range(1, n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.ylim(0, 3.5)
    plt.legend(loc='best')
    plt.show()

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.name = "Net"

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1, stride=2)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1, stride=2)

        # Pooling layer (halves dimensions)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 4 * 4, 256)  # Adjusted for the flattened output
        self.fc2 = nn.Linear(256, 7)  # Number of classes

    def forward(self, x):
        # Applying the first convolutional and pooling layers
        x = self.pool(F.relu(self.conv1(x)))

        # Applying the second convolutional and pooling layers
        x = self.pool(F.relu(self.conv2(x)))

        # Flattening the output for the fully-connected layer
        x = x.view(-1, 16 * 4 * 4)  # Adjust the size accordingly

        # Passing through the fully-connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [14]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, f1, f2, f3, f4, f5, f6):
        super(InceptionBlock, self).__init__()
        self.name = 'InceptionBlock'
        self.conv1 = nn.Conv2d(in_channels, f1, kernel_size=1, padding=0)
        self.conv3_1 = nn.Conv2d(in_channels, f2, kernel_size=1, padding=0)
        self.conv3_2 = nn.Conv2d(f2, f3, kernel_size=3, padding=1)
        self.conv5_1 = nn.Conv2d(in_channels, f4, kernel_size=1, padding=0)
        self.conv5_2 = nn.Conv2d(f4, f5, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.pool_conv = nn.Conv2d(in_channels, f6, kernel_size=1, padding=0)

    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv3_1(x))
        x2 = F.relu(self.conv3_2(x2))
        x3 = F.relu(self.conv5_1(x))
        x3 = F.relu(self.conv5_2(x3))
        x4 = self.pool(x)
        x4 = F.relu(self.pool_conv(x4))
        return torch.cat([x1, x2, x3, x4], dim=1)

class PrimaryModel(nn.Module):
    def __init__(self):
        super(PrimaryModel, self).__init__()
        self.name = 'PrimaryModel'
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.inception1 = InceptionBlock(64, 8, 48, 8, 8, 8, 8)
        
        # First additional convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # Second additional convolutional layer
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(128, 256)  # Adjusted for the output of conv3
        self.fc2 = nn.Linear(256, 128)  # Additional fully connected layer
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 7)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.inception1(x)
        x = F.relu(self.bn2(self.conv2(x)))  # Applying the first new convolutional layer
        x = F.relu(self.bn3(self.conv3(x)))  # Applying the second new convolutional layer
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))  # Additional activation before the final layer
        x = self.fc4(x)
        return x

In [15]:
primary_model = PrimaryModel()
if torch.cuda.is_available():
    primary_model.cuda()

train_net(primary_model, train_loader, val_loader, batch_size=64,learning_rate=0.025,num_epochs=30)

0 initiated
0 batch initiated
1 batch initiated
2 batch initiated
3 batch initiated
4 batch initiated
5 batch initiated
6 batch initiated
7 batch initiated
8 batch initiated
9 batch initiated
10 batch initiated
11 batch initiated
12 batch initiated
13 batch initiated
14 batch initiated
15 batch initiated
16 batch initiated
17 batch initiated
18 batch initiated
19 batch initiated
20 batch initiated
21 batch initiated
22 batch initiated
23 batch initiated
24 batch initiated
25 batch initiated
26 batch initiated
27 batch initiated
28 batch initiated
29 batch initiated
30 batch initiated
31 batch initiated
32 batch initiated
33 batch initiated
34 batch initiated
35 batch initiated
36 batch initiated
37 batch initiated
38 batch initiated
39 batch initiated
40 batch initiated
41 batch initiated
42 batch initiated
43 batch initiated
44 batch initiated
45 batch initiated
46 batch initiated
47 batch initiated
48 batch initiated
49 batch initiated
50 batch initiated
51 batch initiated
52 batch i

77 batch initiated
78 batch initiated
79 batch initiated
80 batch initiated
81 batch initiated
82 batch initiated
83 batch initiated
84 batch initiated
85 batch initiated
86 batch initiated
87 batch initiated
88 batch initiated
89 batch initiated
90 batch initiated
91 batch initiated
92 batch initiated
93 batch initiated
94 batch initiated
95 batch initiated
96 batch initiated
97 batch initiated
98 batch initiated
99 batch initiated
100 batch initiated
101 batch initiated
102 batch initiated
103 batch initiated
104 batch initiated
105 batch initiated
106 batch initiated
107 batch initiated
108 batch initiated
109 batch initiated
Epoch 4: Train err: 0.30313837375178315, Train loss: 0.8267601695927707 | Validation err: 0.3810316139767055, Validation loss: 1.088352563533377
4 initiated
0 batch initiated
1 batch initiated
2 batch initiated
3 batch initiated
4 batch initiated
5 batch initiated
6 batch initiated
7 batch initiated
8 batch initiated
9 batch initiated
10 batch initiated
11 batc

36 batch initiated
37 batch initiated
38 batch initiated
39 batch initiated
40 batch initiated
41 batch initiated
42 batch initiated
43 batch initiated
44 batch initiated
45 batch initiated
46 batch initiated
47 batch initiated
48 batch initiated
49 batch initiated
50 batch initiated
51 batch initiated
52 batch initiated
53 batch initiated
54 batch initiated
55 batch initiated
56 batch initiated
57 batch initiated
58 batch initiated
59 batch initiated
60 batch initiated
61 batch initiated
62 batch initiated
63 batch initiated
64 batch initiated
65 batch initiated
66 batch initiated
67 batch initiated
68 batch initiated
69 batch initiated
70 batch initiated
71 batch initiated
72 batch initiated
73 batch initiated
74 batch initiated
75 batch initiated
76 batch initiated
77 batch initiated
78 batch initiated
79 batch initiated
80 batch initiated
81 batch initiated
82 batch initiated
83 batch initiated
84 batch initiated
85 batch initiated
86 batch initiated
87 batch initiated
88 batch ini

0 batch initiated
1 batch initiated
2 batch initiated
3 batch initiated
4 batch initiated
5 batch initiated
6 batch initiated
7 batch initiated
8 batch initiated
9 batch initiated
10 batch initiated
11 batch initiated
12 batch initiated
13 batch initiated
14 batch initiated
15 batch initiated
16 batch initiated
17 batch initiated
18 batch initiated
19 batch initiated
20 batch initiated
21 batch initiated
22 batch initiated
23 batch initiated
24 batch initiated
25 batch initiated
26 batch initiated
27 batch initiated
28 batch initiated
29 batch initiated
30 batch initiated
31 batch initiated
32 batch initiated
33 batch initiated
34 batch initiated
35 batch initiated
36 batch initiated
37 batch initiated
38 batch initiated
39 batch initiated
40 batch initiated
41 batch initiated
42 batch initiated
43 batch initiated
44 batch initiated
45 batch initiated
46 batch initiated
47 batch initiated
48 batch initiated
49 batch initiated
50 batch initiated
51 batch initiated
52 batch initiated
53 

77 batch initiated
78 batch initiated
79 batch initiated
80 batch initiated
81 batch initiated
82 batch initiated
83 batch initiated
84 batch initiated
85 batch initiated
86 batch initiated
87 batch initiated
88 batch initiated
89 batch initiated
90 batch initiated
91 batch initiated
92 batch initiated
93 batch initiated
94 batch initiated
95 batch initiated
96 batch initiated
97 batch initiated
98 batch initiated
99 batch initiated
100 batch initiated
101 batch initiated
102 batch initiated
103 batch initiated
104 batch initiated
105 batch initiated
106 batch initiated
107 batch initiated
108 batch initiated
109 batch initiated
Epoch 15: Train err: 0.2574893009985735, Train loss: 0.6827106280760331 | Validation err: 0.39001663893510813, Validation loss: 0.9787398513327253
15 initiated
0 batch initiated
1 batch initiated
2 batch initiated
3 batch initiated
4 batch initiated
5 batch initiated
6 batch initiated
7 batch initiated
8 batch initiated
9 batch initiated
10 batch initiated
11 b

35 batch initiated
36 batch initiated
37 batch initiated
38 batch initiated
39 batch initiated
40 batch initiated
41 batch initiated
42 batch initiated
43 batch initiated
44 batch initiated
45 batch initiated
46 batch initiated
47 batch initiated
48 batch initiated
49 batch initiated
50 batch initiated
51 batch initiated
52 batch initiated
53 batch initiated
54 batch initiated
55 batch initiated
56 batch initiated
57 batch initiated
58 batch initiated
59 batch initiated
60 batch initiated
61 batch initiated
62 batch initiated
63 batch initiated
64 batch initiated
65 batch initiated
66 batch initiated
67 batch initiated
68 batch initiated
69 batch initiated
70 batch initiated
71 batch initiated
72 batch initiated
73 batch initiated
74 batch initiated
75 batch initiated
76 batch initiated
77 batch initiated
78 batch initiated
79 batch initiated
80 batch initiated
81 batch initiated
82 batch initiated
83 batch initiated
84 batch initiated
85 batch initiated
86 batch initiated
87 batch ini

0 batch initiated
1 batch initiated
2 batch initiated
3 batch initiated
4 batch initiated
5 batch initiated
6 batch initiated
7 batch initiated
8 batch initiated
9 batch initiated
10 batch initiated
11 batch initiated
12 batch initiated
13 batch initiated
14 batch initiated
15 batch initiated
16 batch initiated
17 batch initiated
18 batch initiated
19 batch initiated
20 batch initiated
21 batch initiated
22 batch initiated
23 batch initiated
24 batch initiated
25 batch initiated
26 batch initiated
27 batch initiated
28 batch initiated
29 batch initiated
30 batch initiated
31 batch initiated
32 batch initiated
33 batch initiated
34 batch initiated
35 batch initiated
36 batch initiated
37 batch initiated
38 batch initiated
39 batch initiated
40 batch initiated
41 batch initiated
42 batch initiated
43 batch initiated
44 batch initiated
45 batch initiated
46 batch initiated
47 batch initiated
48 batch initiated
49 batch initiated
50 batch initiated
51 batch initiated
52 batch initiated
53 

77 batch initiated
78 batch initiated
79 batch initiated
80 batch initiated
81 batch initiated
82 batch initiated
83 batch initiated
84 batch initiated
85 batch initiated
86 batch initiated
87 batch initiated
88 batch initiated
89 batch initiated
90 batch initiated
91 batch initiated
92 batch initiated
93 batch initiated
94 batch initiated
95 batch initiated
96 batch initiated
97 batch initiated
98 batch initiated
99 batch initiated
100 batch initiated
101 batch initiated
102 batch initiated
103 batch initiated
104 batch initiated
105 batch initiated
106 batch initiated
107 batch initiated
108 batch initiated
109 batch initiated
Epoch 26: Train err: 0.23152639087018545, Train loss: 0.6272392104972493 | Validation err: 0.34076539101497505, Validation loss: 0.8697744722061969
26 initiated
0 batch initiated
1 batch initiated
2 batch initiated
3 batch initiated
4 batch initiated
5 batch initiated
6 batch initiated
7 batch initiated
8 batch initiated
9 batch initiated
10 batch initiated
11 

35 batch initiated
36 batch initiated
37 batch initiated
38 batch initiated
39 batch initiated
40 batch initiated
41 batch initiated
42 batch initiated
43 batch initiated
44 batch initiated
45 batch initiated
46 batch initiated
47 batch initiated
48 batch initiated
49 batch initiated
50 batch initiated
51 batch initiated
52 batch initiated
53 batch initiated
54 batch initiated
55 batch initiated
56 batch initiated
57 batch initiated
58 batch initiated
59 batch initiated
60 batch initiated
61 batch initiated
62 batch initiated
63 batch initiated
64 batch initiated
65 batch initiated
66 batch initiated
67 batch initiated
68 batch initiated
69 batch initiated
70 batch initiated
71 batch initiated
72 batch initiated
73 batch initiated
74 batch initiated
75 batch initiated
76 batch initiated
77 batch initiated
78 batch initiated
79 batch initiated
80 batch initiated
81 batch initiated
82 batch initiated
83 batch initiated
84 batch initiated
85 batch initiated
86 batch initiated
87 batch ini

In [16]:
path = get_model_name("PrimaryModel", batch_size=64, learning_rate=0.01, epoch=99)
plot_training_curve(path)

FileNotFoundError: model_PrimaryModel_bs64_lr0.01_epoch99_train_err.csv not found.